In [1]:
from deepface.detectors import SsdWrapper
from deepface.detectors import OpenCvWrapper
from deepface.commons import distance as dst
from deepface.commons import functions
from deepface import DeepFace


import pandas as pd
import numpy as np
import pickle
import time
import cv2

from tensorflow.keras.preprocessing import image

In [2]:
def detect_face(detector, img):

    detected_face = None
    img_region = None
    img_region = [0, 0, img.shape[1], img.shape[0]]

    ssd_labels = ["img_id", "is_face", "confidence", "left", "top", "right", "bottom"]

    target_size = (300, 300)

    base_img = img.copy()  # we will restore base_img to img later

    original_size = img.shape

    img = cv2.resize(img, target_size)

    aspect_ratio_x = original_size[1] / target_size[1]
    aspect_ratio_y = original_size[0] / target_size[0]

    imageBlob = cv2.dnn.blobFromImage(image=img)

    face_detector = detector["face_detector"]
    face_detector.setInput(imageBlob)
    detections = face_detector.forward()

    detections_df = pd.DataFrame(detections[0][0], columns=ssd_labels)

    detections_df = detections_df[detections_df["is_face"] == 1]  # 0: background, 1: face
    detections_df = detections_df[detections_df["confidence"] >= 0.90]

    detections_df["left"] = (detections_df["left"] * 300).astype(int)
    detections_df["bottom"] = (detections_df["bottom"] * 300).astype(int)
    detections_df["right"] = (detections_df["right"] * 300).astype(int)
    detections_df["top"] = (detections_df["top"] * 300).astype(int)

    for _, instance in detections_df.iterrows():

        left = instance["left"]
        right = instance["right"]
        bottom = instance["bottom"]
        top = instance["top"]

        detected_face = base_img[
            int(top * aspect_ratio_y) : int(bottom * aspect_ratio_y),
            int(left * aspect_ratio_x) : int(right * aspect_ratio_x),
        ]
        img_region = [
            int(left * aspect_ratio_x),
            int(top * aspect_ratio_y),
            int(right * aspect_ratio_x) - int(left * aspect_ratio_x),
            int(bottom * aspect_ratio_y) - int(top * aspect_ratio_y),
        ]
        if 0 in detected_face.shape:
            return [None, None]
        
        detected_face = OpenCvWrapper.align_face(detector["eye_detector"], detected_face)
        break

    return [detected_face, img_region]


def extract_faces(img, target_size):
    current_img, current_region = detect_face(SsdWrapper.build_model(), img)
    if current_img is not None:

        # resize and padding
        if current_img.shape[0] > 0 and current_img.shape[1] > 0:
            factor_0 = target_size[0] / current_img.shape[0]
            factor_1 = target_size[1] / current_img.shape[1]
            factor = min(factor_0, factor_1)

            dsize = (int(current_img.shape[1] * factor), int(current_img.shape[0] * factor))
            current_img = cv2.resize(current_img, dsize)

            diff_0 = target_size[0] - current_img.shape[0]
            diff_1 = target_size[1] - current_img.shape[1]
            
            current_img = np.pad(
                current_img,
                (
                    (diff_0 // 2, diff_0 - diff_0 // 2),
                    (diff_1 // 2, diff_1 - diff_1 // 2),
                    (0, 0),
                ),
                "constant",
            )

        # double check: if target image is not still the same size with target.
        if current_img.shape[0:2] != target_size:
            current_img = cv2.resize(current_img, target_size)

        # normalizing the image pixels
        current_img = image.img_to_array(current_img)  # what this line doing? must?
        current_img = np.expand_dims(current_img, axis=0)
        current_img /= 255  # normalize input in [0, 1]

    return [current_img, current_region]

In [ ]:
# paste database path like: "C:/workspace/my_db"
db_path = ""
model_name="Facenet512"
detector_backend="ssd"
distance_metric="cosine"
source=0
target_size = functions.find_target_size(model_name=model_name)

# ------------------------
model = DeepFace.build_model(model_name=model_name)
print(f"facial recognition model {model_name} is just built")

DeepFace.find(
    img_path=np.zeros([400, 400, 3], dtype='uint8'),
    db_path=db_path,
    model_name=model_name,
    detector_backend=detector_backend,
    distance_metric=distance_metric,
    enforce_detection=False,
)
# -----------------------

file_name = f"representations_{model_name}.pkl"
file_name = file_name.replace("-", "_").lower()

with open(f"{db_path}/{file_name}", "rb") as f:
    representations = pickle.load(f)

df = pd.DataFrame(representations, columns=["identity", f"{model_name}_representation"])

In [4]:
cap = cv2.VideoCapture(source)  # webcam
while True:
    tic = time.time()

    _, img = cap.read()
    if img is None:
        break

    raw_img = img.copy()
    target_img, target_region = extract_faces(raw_img, target_size)

    if target_img is not None:
        x, y, w, h = target_region

        if w > 130:  # discard small detected faces

            cv2.rectangle(img, (x, y), (x + w, y + h), (0, 200, 200), 5)

            target_representation = model(target_img, training=False).numpy()[0].tolist()
            result_df = df.copy()  # df will be filtered in each img

            distances = []
            for index, instance in df.iterrows():
                source_representation = instance[f"{model_name}_representation"]

                if distance_metric == "cosine":
                    distance = dst.findCosineDistance(source_representation, target_representation)
                elif distance_metric == "euclidean":
                    distance = dst.findEuclideanDistance(source_representation, target_representation)
                elif distance_metric == "euclidean_l2":
                    distance = dst.findEuclideanDistance(
                        dst.l2_normalize(source_representation),
                        dst.l2_normalize(target_representation),
                    )
                else:
                    raise ValueError(f"invalid distance metric passes - {distance_metric}")

                distances.append(distance)

                # ---------------------------

            result_df[f"{model_name}_{distance_metric}"] = distances

            threshold = dst.findThreshold(model_name, distance_metric)
            result_df = result_df[result_df[f"{model_name}_{distance_metric}"] <= threshold]
            result_df = result_df.sort_values(
                by=[f"{model_name}_{distance_metric}"], ascending=True
            ).reset_index(drop=True)

            if result_df.shape[0] > 0:
                label = result_df.loc[0, "identity"].split("/")[-1]
                img = cv2.putText(img, label.split('.')[0], (10,30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0), 2, cv2.LINE_AA)

                # -------------------------------

    toc = time.time()

    img = cv2.putText(img, str(round(1/(toc-tic), 1)), (img.shape[1]-70,30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,0,0), 2, cv2.LINE_AA)
    cv2.imshow("img", img)

    if cv2.waitKey(1) & 0xFF == ord("q"):  # press q to quit
        break

cap.release()
cv2.destroyAllWindows()